# **Dataset - NBA Players Stats (Classificação)**

O objetivo do grupo ao analisar esse dataset é, por meio das estatisticas dos jogadores da NBA durante a temporada, descobrir qual a posição em que cada um deles joga.

## **1. Inserção das bibliotecas e leitura dos arquivos**

Nessa etapa será feita a leitura dos arquivos (datasets) a serem analisados pelo grupo, além da criação dos dataframes.

#### Imports Python

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None) # Para mostrar todas as colunas no .describe()

#### Leitura dos datasets

In [ ]:
df_regular = pd.read_csv('NBA/2021-2022 NBA Player Stats - Regular.csv', encoding='latin-1', sep=';')
df_playoffs = pd.read_csv('NBA/2021-2022 NBA Player Stats - Playoffs.csv', encoding='latin-1', sep=';')


#### Juntando os dois datasets

In [ ]:
df_total = pd.concat([df_regular, df_playoffs], ignore_index=True)

In [ ]:
df_total.head()

## **2. Analise Exploratória dos Dados**

PLACEHOLDER

O grupo iniciou utilizando a função .info() do dataframe, para verificar contagem total de valores não nulos e o tipo de dado de cada coluna

In [ ]:
df_total.info()

In [ ]:
df_total.describe(include='all') # Descrição do todo

Com base na descrição, é possível ver que há 31 times (a liga possui 30), e um time com sigla TOT. TOT é utilizado para sumarizar os dados de um jogador quando o mesmo jogou por mais de um time na temporada. Além disso, quando o jogador que jogou por mais de um time e jogou em posições distintas ele pode aparecer com uma "posição mista" (como 'SF-PF'). Isso atrapalharia o modelo.

In [ ]:
df_total['Pos'].unique()

Para confirmar a quantidade de posições registradas no dataset, o grupo utilizou a função .unique() na classe alvo. Os resultados mostram que existem posições mistas no dataset. Para checar se são originadas devido as linhas com TOT, iremos removê-las e rodar novamente. -> a remoção final deve vir dps

In [ ]:
posicoes_para_remover = df_total[df_total['Tm'] == 'TOT'].index
df_total = df_total.drop(posicoes_para_remover)
df_total['Pos'].unique()

É possível observar que agora apenas as posições originais do jogo estão no Dataset.

#### Descrevendo classe por classe

O grupo optou pela descrição classe por classe para obter um panorama geral dos dados, segregados por cada classe

In [ ]:
display(df_total.describe())
display(df_total[df_total["Pos"] == "PG"].describe())
display(df_total[df_total["Pos"] == "SG"].describe())
display(df_total[df_total["Pos"] == "SF"].describe())
display(df_total[df_total["Pos"] == "PF"].describe())
display(df_total[df_total["Pos"] == "C"].describe())

Dando seguimento a análise exploratória, o grupo optou por gerar o gráfico de correlação, para identificar o nível de correlação entre os atributos, de forma a aprimorar o resto da EDA. Com base na leitura dos dados é possível observar que as características que marcam tentativas e as que marcam de fato quantos pontos foram feitos (como 3PA e 3P) possuem alta correlação (por falarem da mesma coisa, porém com um ponto de vista diferente). 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_total.corr(numeric_only=True).style.background_gradient(cmap= 'magma')

Com base na análise realizada para os casos onde há o dado para tentativas e acertos, o grupo optou por seguir utilizando apenas as features de tentativa, por possuírem uma distribuição melhor e mais concisa (valores mais dispersos).

In [ ]:
sns.boxplot(df_total, x='Pos', y='3PA', fill=False, color='blue')

In [ ]:
sns.boxplot(df_total, x='Pos', y='3P', fill=False, color='red')

Com base na leitura dos dados, o grupo realizou um gráfico de dispersão com as colunas principais para saber a distribuição dos dados e as correlações entre features com valores plotados.

In [ ]:
teste = df_total[['PTS', 'AST', 'TRB', '3PA', '2PA', 'TOV', 'Pos']]
sns.pairplot(teste, hue='Pos', height=3.5)

Com base na leitura dos dados, o grupo realizou um gráfico para saber quais posições possuem mais PTS (Pontos), AST (Assistências) e TRB (Rebotes). Com base no gráfico, entende-se que a posição C (Pivô) tende a realizar mais rebotes e a posição PG (Armador) tende a realizar mais assistências durante a temporada.

In [ ]:
stats = (
    df_total.groupby('Pos')[['PTS', 'AST', 'TRB']]
    .mean()
    .sort_values('PTS', ascending=False)
)

In [ ]:
plt.figure(figsize=(10,6))
stats.plot(kind='bar', figsize=(10,6), width=0.7, color=['#FF6B6B', '#4ECDC4', '#1A535C'])
plt.title('Média de Estatísticas por Posição - NBA 2021-2022', fontsize=14, pad=15)
plt.xlabel('Posição')
plt.ylabel('Média por Jogo')
plt.legend(['PTS', 'AST', 'TRB'], title='Estatísticas')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

O gráfico mostra a quantidade de jogadores por posição no dataset.

Percebe-se que há mais jogadores SG e menos C, indicando um leve desbalanceamento entre as classes.

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data=df_total, x='Pos', hue='Pos', palette='crest', legend=False)
plt.title('Quantidade de Jogadores por Posição')
plt.xlabel('Posição')
plt.ylabel('Número de Jogadores')
plt.show()